In [1]:
# Imports
import torch, wandb, time
import torch.nn as nn
from torch.utils.data import DataLoader,random_split,SubsetRandomSampler, ConcatDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Internal Imports
from model import AudioClassifier
from util import WavDataSet, SoundDS, PlotSpectrogram, predict, classes, classes_reverse
from training_standard import training, inference  
from training_k_fold import train_epoch, valid_epoch

# GPU Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
# Data Setup

# # Create index
# data = WavDataSet("D:/Documents/Samples/Beat Packs")
# print(data)

# # Save CSV
# data.csv("./data/edm_no_loops.csv")

# # Check Results 
# df = pd.read_csv("./data/edm_no_loops.csv") 
# print(df.head())
# print(f"Dataset Length: {len(df)}")

# Create Pandas Dataframe from CSV
df = pd.read_csv("./data/edm_no_loops.csv")
df = df[['path', 'class']]

# Prepare Batches of Data using the Dataloader
# Random 80:20 Split of Train:Validate
myds = SoundDS(df)
num_items = len(myds)
num_train = round(num_items * 0.67)
num_val = num_items-num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

In [3]:
# Standard Training, Validation and Prediction
# batch_size = 64
# num_epochs = 5

# model = AudioClassifier()
# model.to(device)

# train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
# val_dl = torch.utils.data.DataLoader(val_ds, batch_size=128, shuffle=True)

# training(model, train_dl, device=device, num_epochs=num_epochs, logger=None)
# inference(model, val_dl, device=device, logger=None)

# Make a Prediction
# predict(model, "examples/TR-808Kick01.wav", "kick")

In [4]:
# K Fold Cross Validation
batch_size = 64 
num_epochs = 5 
lr = 3e-4
dataset = ConcatDataset([train_ds, val_ds])

# For Training, use a High Epoch and Low K
# For Evaluation, use a Low Epoch and K of ~10
k = 5
torch.manual_seed(42)
splits = KFold(n_splits=k,shuffle=True,random_state=42)
foldperf = {}

criterion = nn.CrossEntropyLoss()

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    wandb.init(project="beatpack-ai", entity="parabyl", job_type='k-fold', name=f'fold_{fold}') # Running this line creates a new 'run'
    
    start_time = time.time()
    history = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler) 
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = AudioClassifier()
    model.to(device)
    wandb.watch(model, log_freq=100)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Load Previous Model
    # cp = torch.load('models/zesty-salad-125_e512_f3_b128.pt')
    # model.load_state_dict(cp['model_state_dict'])
    # optimizer.load_state_dict(cp['optimizer_state_dict'])
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002, steps_per_epoch=int(len(train_loader)), epochs=num_epochs, anneal_strategy='linear')
    
    print(f"Fold {fold+1}")

    for epoch in range(num_epochs):
        
        train_loss, train_correct = train_epoch(model, device, train_loader, criterion, optimizer, scheduler)
        test_loss, test_correct = valid_epoch(model, device, test_loader, criterion)
        
        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        test_loss = test_loss / len(test_loader.sampler)
        test_acc = test_correct / len(test_loader.sampler) * 100
        
        print(f"Epoch: {epoch+1}/{num_epochs} AVG Training Loss: {train_loss:.3f} AVG Test Loss: {test_loss:.3f} AVG Training Acc: {train_acc:.2f}% AVG Test Acc: {test_acc:.2f}% Time: {time.time() - start_time:.2f}s")
        
        wandb.log({
            'Epoch': epoch+1, 
            'Avg. Training Loss': train_loss, 
            'Avg. Test Loss': test_loss, 
            'Avg. Training Accuracy': train_acc, 
            'Avg. Test Accuracy': test_acc})

               
        history['train_loss'].append(train_loss)
        history['test_loss'].append(test_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        
    foldperf[f'fold{fold}'] = history
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'models/{wandb.run.name}_e{num_epochs}_f{fold}_b{batch_size}.pt')
    wandb.join()

wandb.finish()

# Test Report

testl_f,tl_f,testa_f,ta_f=[],[],[],[]
for f in range(0,k):
    tl_f.append(np.mean(foldperf[f'fold{f}']['train_loss']))
    testl_f.append(np.mean(foldperf[f'fold{f}']['test_loss']))

    ta_f.append(np.mean(foldperf[f'fold{f}']['train_acc']))
    testa_f.append(np.mean(foldperf[f'fold{f}']['test_acc']))

print(f'K={k} fold cross validation complete')
print(f"Avg Training Loss: {np.mean(tl_f):.2f} Avg Test Loss: {np.mean(testl_f):.2f} Avg Training Acc: {np.mean(ta_f)} Avg Test Acc: {np.mean(testa_f)}")     

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: parabyl. Use `wandb login --relogin` to force relogin


Fold 1
Epoch: 1/5 AVG Training Loss: 1.953 AVG Test Loss: 1.712 AVG Training Acc: 32.39% AVG Test Acc: 43.75% Time: 70.16s
Epoch: 2/5 AVG Training Loss: 1.669 AVG Test Loss: 1.599 AVG Training Acc: 42.31% AVG Test Acc: 44.64% Time: 129.92s
Epoch: 3/5 AVG Training Loss: 1.559 AVG Test Loss: 1.520 AVG Training Acc: 44.37% AVG Test Acc: 45.87% Time: 190.17s
Epoch: 4/5 AVG Training Loss: 1.466 AVG Test Loss: 1.501 AVG Training Acc: 46.83% AVG Test Acc: 45.76% Time: 250.42s


KeyboardInterrupt: 